In [1]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 10 12:48:22 2018

@author: lj
"""
import tensorflow as tf
import numpy as np
from math import sqrt

def load_data(file_name):
    '''导入数据
    input:  file_name(string):文件的存储位置
    output: feature_data(mat):特征
            label_data(mat):标签
            n_class(int):类别的个数
    '''
    # 1、获取特征
    f = open(file_name)  # 打开文件
    feature_data = []
    label_data = []
    for line in f.readlines():
        feature_tmp = []
        label_tmp = []
        lines = line.strip().split(",")
        feature_tmp.append(float(lines[1]))
        label_tmp.append(float(lines[0]))      
        feature_data.append(feature_tmp)
        label_data.append(label_tmp)
    f.close()  # 关闭文件

    return np.mat(feature_data), np.mat(label_data)

class RBF_NN():
    def __init__(self,hidden_nodes,input_data_trainX,input_data_trainY):
        self.hidden_nodes = hidden_nodes #隐含层节点数
        self.input_data_trainX = input_data_trainX #训练样本的特征
        self.input_data_trainY = input_data_trainY #训练样本的标签
    
    def fit(self):
        '''模型训练
        '''
        n_input = (self.input_data_trainX).shape[1]
        n_output = (self.input_data_trainY).shape[1]
        
        #fc = open("rbf/c",'r')
        #cen = []
        #for line in fc.readlines():
        #    lines = line.strip() 
        #    cen.append(float(lines))
        #fc.close()
        #delta = (cen[-1] - cen[0]) / sqrt(2 * len(cen))
        #c = tf.convert_to_tensor(cen,dtype=tf.float32)
        #c = tf.reshape(c,[self.hidden_nodes,n_input])
        
        c = np.array(range(20,40)) * 0.01
        delta = 20 / sqrt(2 * 20)
        
        X = tf.placeholder('float',[None,n_input],name = 'X')
        Y = tf.placeholder('float',[None,n_output],name = 'Y')
        # 2.参数设置
        ## RBF函数参数
        c = tf.Variable(tf.random_normal([self.hidden_nodes,n_input]),name = 'c')
        delta = tf.Variable(tf.random_normal([1,self.hidden_nodes]),name = 'delta')
        ## 隐含层到输出层权重和偏置
        W = tf.Variable(tf.random_normal([self.hidden_nodes,n_output]),name = 'W')
        b = tf.Variable(tf.random_normal([1,n_output]),name = 'b')
        # 3.构造前向传播计算图
        ## 隐含层输出
        ### 特征样本与RBF均值的距离
        dist = tf.reduce_sum(tf.square(tf.subtract(tf.tile(X,[self.hidden_nodes,1]),c)),1) 
        dist = tf.reshape(dist,[self.hidden_nodes,n_output])
        dist = tf.multiply(1.0,tf.transpose(dist))
        ### RBF方差的平方
        delta_2 = -1.0 / (2.0 * delta * delta)
        ### 隐含层输出
        RBF_OUT = tf.exp(dist * delta_2)        
        ## 输出层输入
        output_in = tf.matmul(RBF_OUT,W) + b
        ## 输出层输出
        y_pred = output_in
        # 4.声明代价函数优化算法
        cost = tf.reduce_mean(tf.pow(Y - y_pred,2)) #损失函数为均方误差
        #train_op = tf.train.GradientDescentOptimizer(0.05).minimize(cost) #优化算法为梯度下降法
        train_op = tf.train.AdamOptimizer(0.001).minimize(cost)
        
        # 5.反向传播求参数
        trX = self.input_data_trainX
        trY = self.input_data_trainY
        
        with tf.Session() as sess:
            ##初始化所有参数
            tf.global_variables_initializer().run()
            for epoch in range(100):
                loss = 0
                for i in range(len(trX)):
                    feed = {X:trX[i],Y:trY[i]}
                    _, l = sess.run([train_op,cost],feed_dict = feed)
                    loss += l
                print('epoch:%d, Loss function is %f'%(epoch, loss))
            print('Training complete!')

            W = W.eval()
            b = b.eval()
            c = c.eval()
            delta = delta.eval()
            print(c)
            ## 训练准确率
            for i in range(len(trX)):
                pred_tr = sess.run(y_pred,feed_dict = {X:trX[i]})
                print("input:%f, output:%f, predict:%f"%(trX[i],trY[i],pred_tr))
            
    def save_model(self,file_name,weights):
        '''保存模型(保存权重weights)
        input：file_name(string):文件名
               weights(mat)：权重矩阵
        '''
        f_w = open(file_name,'w')
        m,n = np.shape(weights)
        for i in range(m):
            w_tmp = []
            for j in range(n):
                w_tmp.append(str(weights[i,j]))
            f_w.write('\t'.join(w_tmp)+'\n')
        f_w.close()
            
if __name__ == '__main__':
    print('------------------------1.Load Data---------------------')
    trainX, trainY = load_data('rbf/train')
    print('------------------------2.parameter setting-------------')
    hidden_nodes = 20
    input_data_trainX = trainX
    input_data_trainY = trainY
    rbf = RBF_NN(hidden_nodes,input_data_trainX,input_data_trainY)
    rbf.fit()

------------------------1.Load Data---------------------
------------------------2.parameter setting-------------
epoch:0, Loss function is 60.946249
epoch:1, Loss function is 57.879429
epoch:2, Loss function is 57.639997
epoch:3, Loss function is 57.514915
epoch:4, Loss function is 57.385013
epoch:5, Loss function is 57.186529
epoch:6, Loss function is 56.932625
epoch:7, Loss function is 56.642452
epoch:8, Loss function is 56.340218
epoch:9, Loss function is 56.033396
epoch:10, Loss function is 55.729167
epoch:11, Loss function is 55.408209
epoch:12, Loss function is 55.075276
epoch:13, Loss function is 54.747488
epoch:14, Loss function is 54.416620
epoch:15, Loss function is 54.108791
epoch:16, Loss function is 53.849494
epoch:17, Loss function is 53.650629
epoch:18, Loss function is 53.514649
epoch:19, Loss function is 53.435863
epoch:20, Loss function is 53.379383
epoch:21, Loss function is 53.318005
epoch:22, Loss function is 53.247418
epoch:23, Loss function is 53.170506
epoch:24

input:0.280000, output:1.437100, predict:1.320515
input:0.290000, output:1.437100, predict:1.360149
input:0.300000, output:1.438800, predict:1.400756
input:0.310000, output:1.481500, predict:1.440987
input:0.330000, output:1.492500, predict:1.515998
input:0.340000, output:1.540400, predict:1.549274
input:0.350000, output:1.542400, predict:1.579179
input:0.360000, output:1.589400, predict:1.605610
input:0.370000, output:1.597900, predict:1.628657
input:0.380000, output:1.595700, predict:1.648557
input:0.390000, output:1.643800, predict:1.665641
input:0.240000, output:1.291700, predict:1.200638
input:0.250000, output:1.287600, predict:1.222117
input:0.260000, output:1.282100, predict:1.250065
input:0.270000, output:1.201200, predict:1.283325
input:0.280000, output:1.387300, predict:1.320515
input:0.290000, output:1.384100, predict:1.360149
input:0.300000, output:1.355900, predict:1.400756
input:0.310000, output:1.395300, predict:1.440987
input:0.320000, output:1.438800, predict:1.479698


input:0.270000, output:0.871500, predict:1.283325
input:0.280000, output:1.048500, predict:1.320515
input:0.290000, output:1.054000, predict:1.360149
input:0.300000, output:1.438800, predict:1.400756
input:0.310000, output:1.512300, predict:1.440987
input:0.320000, output:1.593600, predict:1.479698
input:0.330000, output:1.593600, predict:1.515998
input:0.350000, output:1.739100, predict:1.579179
input:0.360000, output:1.659800, predict:1.605610
input:0.370000, output:1.656300, predict:1.628657
input:0.380000, output:1.584200, predict:1.648557
input:0.390000, output:1.581000, predict:1.665641
input:0.230000, output:1.015200, predict:1.186447
input:0.240000, output:0.969700, predict:1.200638
input:0.250000, output:0.941200, predict:1.222117
input:0.260000, output:0.928100, predict:1.250065
input:0.270000, output:0.924900, predict:1.283325
input:0.280000, output:1.049900, predict:1.320515
input:0.290000, output:1.055400, predict:1.360149
input:0.300000, output:1.438800, predict:1.400756


input:0.340000, output:1.673600, predict:1.549274
input:0.350000, output:1.659800, predict:1.579179
input:0.360000, output:1.826500, predict:1.605610
input:0.370000, output:1.673600, predict:1.628657
input:0.380000, output:1.818200, predict:1.648557
input:0.390000, output:1.826500, predict:1.665641
input:0.240000, output:1.250000, predict:1.200638
input:0.250000, output:1.498100, predict:1.222117
input:0.260000, output:1.532600, predict:1.250065
input:0.270000, output:1.538500, predict:1.283325
input:0.280000, output:1.355900, predict:1.320515
input:0.290000, output:1.333300, predict:1.360149
input:0.300000, output:1.215800, predict:1.400756
input:0.310000, output:1.459900, predict:1.440987
input:0.320000, output:1.498100, predict:1.479698
input:0.330000, output:1.652900, predict:1.515998
input:0.340000, output:1.673600, predict:1.549274
input:0.350000, output:1.666700, predict:1.579179
input:0.360000, output:1.834900, predict:1.605610
input:0.370000, output:1.680700, predict:1.628657


input:0.250000, output:1.230800, predict:1.222117
input:0.260000, output:1.314300, predict:1.250065
input:0.270000, output:1.168500, predict:1.283325
input:0.280000, output:1.135300, predict:1.320515
input:0.290000, output:1.155000, predict:1.360149
input:0.300000, output:1.366700, predict:1.400756
input:0.310000, output:1.366700, predict:1.440987
input:0.320000, output:1.369900, predict:1.479698
input:0.330000, output:1.408500, predict:1.515998
input:0.350000, output:1.574800, predict:1.579179
input:0.360000, output:1.729100, predict:1.605610
input:0.370000, output:1.793700, predict:1.628657
input:0.380000, output:1.857600, predict:1.648557
input:0.390000, output:1.932400, predict:1.665641
input:0.230000, output:0.892900, predict:1.186447
input:0.240000, output:1.223200, predict:1.200638
input:0.250000, output:1.229500, predict:1.222117
input:0.260000, output:1.273900, predict:1.250065
input:0.270000, output:1.314300, predict:1.283325
input:0.280000, output:1.320100, predict:1.320515


plt.rc('font',family='Times New Roman',size=10)
plt.xlabel("iteration number")
plt.ylabel("loss value")
plt.title("train loss")
plt.plot(result)
plt.ylim(0,50)
plt.xlim(20000,200000)